In [1]:
from IPython.display import display, HTML

# Import Concordia framework components
from concordia.language_model.model_client_initialization import ModelClient
import concordia.prefabs.game_master as game_master_prefabs
from concordia.type_checks import prefab as prefab_lib
from concordia.utils import helper_functions

# Import our custom patterns
from concordia.slim_interactive.slim_engine import LivingSimulation
from concordia.slim_interactive.slim_agent import SlimAgent

c:\Users\tyler\miniconda3\envs\a0\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
client = ModelClient(stream=True)
model = client.model
embedder = client.embedder

Initializing model for provider: lmstudio


In [4]:
prefabs = {
    'slim_agent__SlimAgent': SlimAgent(),
    **helper_functions.get_package_classes(game_master_prefabs),
}

# Define the two agents for our initial scenario
PLAYER_ONE_NAME = 'Alex'
PLAYER_TWO_NAME = 'Ben'

player_one_memory = 'I remember seeing a sequence of colors: red, blue, green.'
player_two_memory = 'I remember hearing a sequence of numbers: 4, 8, 2.'

instances = [
    prefab_lib.InstanceConfig(
        prefab='slim_agent__SlimAgent',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_ONE_NAME,
            'goal': 'Work with Ben to figure out how to escape this room.',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='slim_agent__SlimAgent',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_TWO_NAME,
            'goal': 'Work with Alex to figure out how to escape this room.',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='generic__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={'name': 'slim_master'},
    ),
    # This initializer GM will give the agents their initial memory fragments.
    prefab_lib.InstanceConfig(
        prefab='formative_memories_initializer__GameMaster',
        role=prefab_lib.Role.INITIALIZER,
        params={
            'name': 'Memory Initializer',
            'next_game_master_name': 'slim_master',
            'player_specific_memories': {
                PLAYER_ONE_NAME: [player_one_memory],
                PLAYER_TWO_NAME: [player_two_memory],
            },
        },
    ),
]

# Create the main simulation configuration object
config = prefab_lib.Config(
    default_premise=(
        f'{PLAYER_ONE_NAME} and {PLAYER_TWO_NAME} wake up in a locked room. '
        'The room is sparse, containing only a wooden table and two chairs. '
        'There is a single, heavy oak door with no visible keyhole.'
    ),
    prefabs=prefabs,
    instances=instances,
)


In [5]:
runnable_simulation = LivingSimulation(
    config=config,
    model=model,
    embedder=embedder,
)

In [6]:
cycle1_log = runnable_simulation.run_cycle()
display(HTML(cycle1_log))


Terminate? No
Game master: Memory Initializer

Skipping the action phase for the current time step.

Entity Alex observed: When Alex was 5 years old, she experienced her first trip outside the city. It was a camping trip with her parents to a national park, and everything felt magical. The towering trees, the babbling brook, and the starry night sky filled her with wonder.  Alex had never seen so many stars before, and she spent hours lying on her back, tracing constellations with her finger. This trip ignited a lifelong love for nature and exploration within her.





When Alex was 10 years old, she witnessed a heated argument between her parents. They were arguing about money, and the tension in the air felt suffocating. Alex had never seen her usually jovial parents so angry, and it scared her.  She retreated to her room, burying herself in a book, wishing she could escape the harsh reality unfolding before her. This experience instilled in Alex a deep empathy for others struggling 